In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.6.0
GPU is available


In [ ]:
# fashion_mnist = tf.keras.datasets.imdb

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'aflw2k3d',
 'amazon_us_reviews',
 'bair_robot_pushing_small',
 'bigearthnet',
 'binarized_mnist',
 'binary_alpha_digits',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_corrupted',
 'clevr',
 'cnn_dailymail',
 'coco',
 'coco2014',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eurosat',
 'fashion_mnist',
 'flores',
 'food101',
 'gap',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'image_label_folder',
 'imagenet2012',
 'imagenet2012_corrupted',
 'imdb_reviews',
 'iris',
 'kitti',
 'kmnist',
 'lfw',
 'lm1b',
 'lsun',
 'mnist',
 'mnist_corrupted',
 'moving_mnist',
 'multi_nli',
 'nsynth',
 'omniglot',

In [4]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\30660\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=219, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [6]:
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [7]:
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=402, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================].7356 - accuracy: 0.56 - 1s 725ms/step - loss: 0.7413 - accuracy: 0.555 - 2s 508ms/step - loss: 0.7380 - accuracy: 0.558 - 2s 403ms/step - loss: 0.7443 - accuracy: 0.545 - 2s 335ms/step - loss: 0.7411 - accuracy: 0.544 - 2s 289ms/step - loss: 0.7417 - accuracy: 0.540 - 2s 257ms/step - loss: 0.7438 - accuracy: 0.536 - 2s 232ms/step - loss: 0.7402 - accuracy: 0.538 - 2s 214ms/step - loss: 0.7395 - accuracy: 0.537 - 2s 199ms/step - loss: 0.7383 - accuracy: 0.536 - 2s 186ms/step - loss: 0.7351 - accuracy: 0.537 - 2s 176ms/step - loss: 0.7361 - accuracy: 0.535 - 2s 167ms/step - loss: 0.7318 - accuracy: 0.541 - 2s 159ms/step - loss: 0.7283 - accuracy: 0.543 - 2s 153ms/step - loss: 0.7270 - accuracy: 0.545 - 2s 147ms/step - loss: 0.7251 - accuracy: 0.547 - 2s 142ms/step - loss: 0.7226 - accuracy: 0.550 - 2s 137ms/step - loss: 0.7202 - accuracy: 0.552 - 3s 133ms/step - loss: 0.7181 - accuracy: 0.554 - 3s 129ms/step - loss: 0.7151 - accuracy: 0.5

Epoch 9/20
30/30 [==============================] - ETA: 10s - loss: 0.4071 - accuracy: 0.834 - ETA: 5s - loss: 0.4056 - accuracy: 0.823 - ETA: 4s - loss: 0.3921 - accuracy: 0.83 - ETA: 3s - loss: 0.3896 - accuracy: 0.83 - ETA: 2s - loss: 0.3867 - accuracy: 0.83 - ETA: 2s - loss: 0.3861 - accuracy: 0.83 - ETA: 1s - loss: 0.3867 - accuracy: 0.83 - ETA: 1s - loss: 0.3829 - accuracy: 0.83 - ETA: 1s - loss: 0.3817 - accuracy: 0.84 - ETA: 1s - loss: 0.3779 - accuracy: 0.84 - ETA: 1s - loss: 0.3765 - accuracy: 0.84 - ETA: 0s - loss: 0.3755 - accuracy: 0.84 - ETA: 0s - loss: 0.3740 - accuracy: 0.84 - ETA: 0s - loss: 0.3721 - accuracy: 0.84 - ETA: 0s - loss: 0.3707 - accuracy: 0.84 - ETA: 0s - loss: 0.3689 - accuracy: 0.84 - ETA: 0s - loss: 0.3688 - accuracy: 0.84 - ETA: 0s - loss: 0.3679 - accuracy: 0.84 - ETA: 0s - loss: 0.3661 - accuracy: 0.85 - 4s 125ms/step - loss: 0.3651 - accuracy: 0.8503 - val_loss: 0.3849 - val_accuracy: 0.8310
Epoch 10/20
30/30 [==============================] - ETA:

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3212 - accuracy: 0.8595
loss: 0.321
accuracy: 0.860


In [12]:
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/nnlm-en-dim50/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=4995, shape=(3, 50), dtype=float32, numpy=
array([[ 1.4732717 ,  0.42393148,  0.16215092,  0.5873862 ,  0.1513372 ,
        -0.5062546 , -0.21033262, -0.33339503, -0.7381036 ,  0.29208365,
        -0.35330555, -0.02719353, -0.25117522,  0.28229505, -0.05980197,
        -0.73671365, -0.55260205,  0.08691197,  0.08244086, -1.3579206 ,
         0.3993887 , -0.5992716 ,  0.73926276,  0.64969933, -0.32441717,
         0.33525264, -1.1349468 ,  0.1502967 ,  0.8454977 , -1.1183991 ,
        -0.8927187 ,  0.9534588 ,  0.22556058, -0.66705996, -0.65366906,
         0.46913755,  0.5412825 , -0.20910901,  0.31495017, -0.91272557,
         0.02957173,  0.1557533 , -0.6307922 ,  0.43090525, -0.46083456,
        -1.018203  , -0.44839182, -0.35512343, -0.41938427,  0.10584122],
       [ 0.42595312,  0.41425827,  0.05380295,  0.65693736,  0.02124309,
        -0.34377313,  0.27501962, -0.29560134, -0.8899341 ,  0.4253666 ,
        -0.03075071,  0.15959139,  0.05135492,  0.41358554, -0.18

In [13]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_2 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================].7146 - accuracy: 0.51 - 1s 643ms/step - loss: 0.7101 - accuracy: 0.520 - 1s 466ms/step - loss: 0.7071 - accuracy: 0.518 - 2s 376ms/step - loss: 0.7033 - accuracy: 0.523 - 2s 321ms/step - loss: 0.7010 - accuracy: 0.520 - 2s 283ms/step - loss: 0.6985 - accuracy: 0.529 - 2s 256ms/step - loss: 0.6960 - accuracy: 0.535 - 2s 237ms/step - loss: 0.6945 - accuracy: 0.537 - 2s 221ms/step - loss: 0.6928 - accuracy: 0.542 - 2s 208ms/step - loss: 0.6907 - accuracy: 0.550 - 2s 197ms/step - loss: 0.6897 - accuracy: 0.552 - 2s 187ms/step - loss: 0.6884 - accuracy: 0.555 - 2s 180ms/step - loss: 0.6863 - accuracy: 0.562 - 2s 173ms/step - loss: 0.6850 - accuracy: 0.567 - 3s 167ms/step - loss: 0.6831 - accuracy: 0.574 - 3s 163ms/step - loss: 0.6812 - accuracy: 0.581 - 3s 158ms/step - loss: 0.6796 - accuracy: 0.588 - 3s 155ms/step - loss: 0.6780 - accuracy: 0.594 - 3s 151ms/step - loss: 0.6761 - accuracy: 0.598 - 3s 148ms/step - loss: 0.6747 - accuracy: 0.6

Epoch 9/20
30/30 [==============================] - ETA: 11s - loss: 0.0800 - accuracy: 0.988 - ETA: 6s - loss: 0.0794 - accuracy: 0.987 - ETA: 4s - loss: 0.0831 - accuracy: 0.98 - ETA: 3s - loss: 0.0830 - accuracy: 0.98 - ETA: 3s - loss: 0.0791 - accuracy: 0.98 - ETA: 2s - loss: 0.0784 - accuracy: 0.98 - ETA: 2s - loss: 0.0800 - accuracy: 0.98 - ETA: 1s - loss: 0.0809 - accuracy: 0.98 - ETA: 1s - loss: 0.0783 - accuracy: 0.98 - ETA: 1s - loss: 0.0783 - accuracy: 0.98 - ETA: 1s - loss: 0.0784 - accuracy: 0.98 - ETA: 1s - loss: 0.0776 - accuracy: 0.98 - ETA: 0s - loss: 0.0770 - accuracy: 0.98 - ETA: 0s - loss: 0.0764 - accuracy: 0.98 - ETA: 0s - loss: 0.0758 - accuracy: 0.98 - ETA: 0s - loss: 0.0752 - accuracy: 0.98 - ETA: 0s - loss: 0.0753 - accuracy: 0.98 - ETA: 0s - loss: 0.0748 - accuracy: 0.98 - ETA: 0s - loss: 0.0741 - accuracy: 0.98 - 5s 157ms/step - loss: 0.0738 - accuracy: 0.9876 - val_loss: 0.3039 - val_accuracy: 0.8758
Epoch 10/20
30/30 [==============================] - ETA:

In [16]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.4661 - accuracy: 0.8483
loss: 0.466
accuracy: 0.848


In [26]:
# embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/nnlm-en-dim128/1"
embedding = r"C:\model\tfhub_modules\5cf6e1111d63a20038f955024d33f59b34048893"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3]).shape

TensorShape([3, 50])

In [27]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_6 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [29]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================].7130 - accuracy: 0.45 - 1s 600ms/step - loss: 0.7028 - accuracy: 0.498 - 1s 433ms/step - loss: 0.7016 - accuracy: 0.496 - 1s 351ms/step - loss: 0.6992 - accuracy: 0.500 - 1s 300ms/step - loss: 0.6962 - accuracy: 0.506 - 2s 265ms/step - loss: 0.6950 - accuracy: 0.512 - 2s 240ms/step - loss: 0.6936 - accuracy: 0.519 - 2s 222ms/step - loss: 0.6923 - accuracy: 0.525 - 2s 208ms/step - loss: 0.6901 - accuracy: 0.535 - 2s 196ms/step - loss: 0.6889 - accuracy: 0.538 - 2s 186ms/step - loss: 0.6875 - accuracy: 0.543 - 2s 177ms/step - loss: 0.6859 - accuracy: 0.550 - 2s 171ms/step - loss: 0.6840 - accuracy: 0.557 - 2s 164ms/step - loss: 0.6824 - accuracy: 0.561 - 2s 159ms/step - loss: 0.6805 - accuracy: 0.568 - 2s 154ms/step - loss: 0.6785 - accuracy: 0.575 - 3s 150ms/step - loss: 0.6780 - accuracy: 0.578 - 3s 147ms/step - loss: 0.6768 - accuracy: 0.580 - 3s 144ms/step - loss: 0.6751 - accuracy: 0.586 - 3s 141ms/step - loss: 0.6736 - accuracy: 0.5

Epoch 9/20
30/30 [==============================] - ETA: 11s - loss: 0.0677 - accuracy: 0.996 - ETA: 6s - loss: 0.0674 - accuracy: 0.993 - ETA: 4s - loss: 0.0702 - accuracy: 0.99 - ETA: 3s - loss: 0.0705 - accuracy: 0.98 - ETA: 3s - loss: 0.0743 - accuracy: 0.98 - ETA: 2s - loss: 0.0717 - accuracy: 0.98 - ETA: 2s - loss: 0.0711 - accuracy: 0.98 - ETA: 1s - loss: 0.0711 - accuracy: 0.98 - ETA: 1s - loss: 0.0730 - accuracy: 0.98 - ETA: 1s - loss: 0.0734 - accuracy: 0.98 - ETA: 1s - loss: 0.0730 - accuracy: 0.98 - ETA: 1s - loss: 0.0732 - accuracy: 0.98 - ETA: 0s - loss: 0.0733 - accuracy: 0.98 - ETA: 0s - loss: 0.0719 - accuracy: 0.98 - ETA: 0s - loss: 0.0712 - accuracy: 0.98 - ETA: 0s - loss: 0.0704 - accuracy: 0.98 - ETA: 0s - loss: 0.0706 - accuracy: 0.98 - ETA: 0s - loss: 0.0714 - accuracy: 0.98 - ETA: 0s - loss: 0.0712 - accuracy: 0.98 - 5s 150ms/step - loss: 0.0706 - accuracy: 0.9884 - val_loss: 0.3073 - val_accuracy: 0.8765
Epoch 10/20
30/30 [==============================] - ETA:

In [30]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.4866 - accuracy: 0.8472
loss: 0.487
accuracy: 0.847
